# Классификация клиентов телеком компании

В вашем распоряжении данные о поведении клиентов, которые уже перешли на необходимые тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

## Задача

Постройте модель машинного обучения с максимально большим значением *accuracy*. Необходимо довести долю правильных ответов минимум до 0.75.

## Загрузка библиотек

In [1]:
# Датафреймы и математика
import pandas as pd
import math

# Визуализация
import seaborn as sns
import matplotlib.pyplot as plt

# Машинное обучение
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

##  Загрузка датасета

In [2]:
df = pd.read_csv('datasets/users_behavior.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


В наборе данных 3214 объектов с 5-ю признаками:

* `сalls` — количество звонков,
* `minutes` — суммарная длительность звонков в минутах,
* `messages` — количество sms-сообщений,
* `mb_used` — израсходованный интернет-трафик в Мб,
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Предобработка

### Пропуски и дубликаты
Проверим чистоту данных:

In [3]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [4]:
df.duplicated().sum()

0

Пропуски и дубликаты отстутствуют

### Приведение типов
В 4-х из 5-ти столбцов данные представлены типом `float64`. Сразу приведём столбцы `calls` и `messages` к типу `int`:

In [5]:
df[['calls', 'messages']] = df[['calls', 'messages']].astype(int)

В столбцах `minutes` и `mb_used` воспользуемся округлением в большую сторону и тоже приведём к типу `int`, чтобы сократить вычислительную нагрузку:

In [6]:
df['minutes'] = df['minutes'].apply(lambda x: int(math.ceil(x)))
df['mb_used'] = df['mb_used'].apply(lambda x: int(math.ceil(x)))

df.info()
df.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   calls     3214 non-null   int64
 1   minutes   3214 non-null   int64
 2   messages  3214 non-null   int64
 3   mb_used   3214 non-null   int64
 4   is_ultra  3214 non-null   int64
dtypes: int64(5)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40,312,83,19916,0


### Итоги этапа
Данные загружены, предобработаны и готовы для дальнейшей работы.

## Разделение данных на выборки

Делим данные на обучающую, валидационную и тестовую выборки:

In [7]:
# Тестовая выборка
df_train, df_test = train_test_split(df, test_size=0.25, random_state=42)
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

# Валидационная выборка
df_train, df_valid = train_test_split(df_train, test_size=0.25, random_state=42)
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

# Провекра
print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)
print(features_valid.shape)
print(target_valid.shape)

(2410, 4)
(2410,)
(804, 4)
(804,)
(603, 4)
(603,)


## Исследование моделей

### Дерево решений
Обучим 10 моделей с различной глубиной `max_depth` и определим, какая даст самый высокй показатель *accuracy*:

In [8]:
%%time
# Пустая переменная для хранения наилучшей модели
dt_model = None

# Глобальные переменные для лучших показателей
best_accuracy = 0
best_depth = 0

# Цикл по глубине дерева от 1 до 10
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345,
                                   max_depth=depth)
    model.fit(features_train, target_train)
    
    # Проверка accuracy
    accuracy = model.score(features_valid, target_valid)
    if accuracy > best_accuracy:
        dt_model = model
        best_accuracy = accuracy
        best_depth = depth
    
    print("Depth:", depth, "| Accuracy: ", round(accuracy, 3))

print("\nНаилучшая модель с глубиной:", best_depth, "\nAccuracy: ", round(best_accuracy, 3))

Depth: 1 | Accuracy:  0.745
Depth: 2 | Accuracy:  0.765
Depth: 3 | Accuracy:  0.784
Depth: 4 | Accuracy:  0.791
Depth: 5 | Accuracy:  0.818
Depth: 6 | Accuracy:  0.829
Depth: 7 | Accuracy:  0.837
Depth: 8 | Accuracy:  0.856
Depth: 9 | Accuracy:  0.864
Depth: 10 | Accuracy:  0.864

Наилучшая модель с глубиной: 9 
Accuracy:  0.864
CPU times: user 80.3 ms, sys: 3.9 ms, total: 84.2 ms
Wall time: 84.5 ms


### Случайный лес
Обучим 10 моделей с количеством деревьев от 1 до 10 и проверим, какая даст лучший показатель *accuracy*:

In [9]:
%%time
rf_model = None
best_accuracy = 0
best_est = 0

for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345,
                                   n_estimators=est)
    model.fit(features_train, target_train) 
    
    accuracy = model.score(features_valid, target_valid)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        rf_model = model
        best_est = est
    
    print("Trees:", est, "| Accuracy: ", round(accuracy, 3))

print("\nНаилучшая модель с количеством деревьев: :", best_est, "\nAccuracy: ", round(best_accuracy, 3))

Trees: 1 | Accuracy:  0.887
Trees: 2 | Accuracy:  0.907
Trees: 3 | Accuracy:  0.964
Trees: 4 | Accuracy:  0.964
Trees: 5 | Accuracy:  0.97
Trees: 6 | Accuracy:  0.972
Trees: 7 | Accuracy:  0.975
Trees: 8 | Accuracy:  0.978
Trees: 9 | Accuracy:  0.985
Trees: 10 | Accuracy:  0.98

Наилучшая модель с количеством деревьев: : 9 
Accuracy:  0.985
CPU times: user 365 ms, sys: 11.5 ms, total: 376 ms
Wall time: 381 ms


### Логистическая регрессия
Обучим 10 моделей с максимальным количеством итераци от 100 до 1000 (с шагом 100) и проверим, какая даст лучший показатель accuracy:

In [10]:
%%time

lr_model = None
best_accuracy = 0
best_iter = 0

for i in range(100, 1000, 100):
    model = LogisticRegression(random_state=12345,
                               solver='lbfgs',
                               max_iter=i)
    model.fit(features_train, target_train) 
    
    accuracy = model.score(features_valid, target_valid)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        lr_model = model
        best_iter = 100
    
    print("Iterations:", i, "| Accuracy: ", round(accuracy, 9))

print("\nНаилучшая модель с количеством итераций: :", best_iter, "\nAccuracy: ", round(best_accuracy, 9))

Iterations: 100 | Accuracy:  0.751243781
Iterations: 200 | Accuracy:  0.751243781
Iterations: 300 | Accuracy:  0.751243781
Iterations: 400 | Accuracy:  0.751243781
Iterations: 500 | Accuracy:  0.751243781
Iterations: 600 | Accuracy:  0.751243781
Iterations: 700 | Accuracy:  0.751243781
Iterations: 800 | Accuracy:  0.751243781
Iterations: 900 | Accuracy:  0.751243781

Наилучшая модель с количеством итераций: : 100 
Accuracy:  0.751243781
CPU times: user 517 ms, sys: 181 ms, total: 698 ms
Wall time: 181 ms


### Итоги этапа
* Лучшие модели сохранены в переменных: `dt_model`, `rf_model` и `lr_model`
* Самый высокий показатель *accuracy* = 0.983 наблюдается у модели случайного леса с количеством деревьев равным 9-ти
* Accuracy дерева решений возрастает с увеличением количества ветвей
* Линейная регрессия показала самую низкую точность при самых больших временных затратах

Для дальнейшего анализа будем использоавть модель случайного леса с 9-ю деревьями, которая сохраненва в переменной `rf_model`

## Проверка модели на тестовой выборке

### Тестирование
Проверим модель случайного леса с 9-ю дервеьями на тестовой выборке:

In [11]:
accuracy = rf_model.score(features_test, target_test)
print("Accuracy =", round(accuracy, 5))

Accuracy = 0.80224


###  Итоги тестирования
Доля правильных ответов на тестовой выборке выше 0,75, требовавшихся по заданию. Следовательно, задача выполнена.

### Тестирование на модели без валидационной выборки

In [12]:
# Отделяем тестовую выборку
df_train, df_test = train_test_split(df, test_size=0.25, random_state=42)
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

# Повторная тренировка модели
model = RandomForestClassifier(random_state=12345,
                               n_estimators=9) # берём 9 деревьев по результатам предыдущего этапа
model.fit(features_train, target_train)

# Проверка
accuracy = model.score(features_test, target_test)
print("Accuracy =", round(accuracy, 5))

Accuracy = 0.80224


## Проверка модели на адекватность

Воспользуемся `DummyClassifier`

In [13]:
dc_model = DummyClassifier(strategy='most_frequent', random_state=12345)
dc_model.fit(features_train, target_train)

accuracy = dc_model.score(features_valid, target_valid)
print('Accuracy:', accuracy)

Accuracy: 0.6882255389718076


###  Итоги проверки
Показатель *accuracy* для `DummyClassifier` ниже 0,7. Все рассмотренные модели давали показатель выше, следовательно проверка моделей на адекватность пройдена.

# Выводы 

Задачей проекта было построение модеи машинного обучения с максимально большим значением accuracy минимум 0.75.

В ходе анализа были исследованы 3 типа моделей:
* Дерево решений
* Случайны лес
* Логистическая регрессия

Проверку на адекватность прошли все модели.

Наиболее качественный результат показала модель случайного леса с 9-ю деревьями, которая при проверке на тестовой выборке достигла показателя accuracy 0.802.